In [25]:
import re
import os

def list_all_files(root, pattern=None):
    tif_files = []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if not f.lower().endswith(".tif"):
                continue

            full = os.path.join(dirpath, f)

            if pattern and not re.search(pattern, full):
                continue

            tif_files.append(full)

    return tif_files


In [30]:
root = 'DATA_SV/Precipitation/RADAR/2020'
# root = 'DATA_SV/Hima'
# root = 'DATA_SV/ERA5'

files = list_all_files(root)

print("Số file tìm thấy:", len(files))

Số file tìm thấy: 1270


In [34]:
import os
import re

def list_all_files(root, pattern=None):
    """
    Tìm tất cả các file .tif trong thư mục gốc (root) và các thư mục con.
    Lọc các file theo một regex pattern tùy chọn.
    """
    tif_files = []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            # 1. Lọc file .tif
            if not f.lower().endswith(".tif"):
                continue

            full = os.path.join(dirpath, f)

            # 2. Lọc theo pattern (nếu pattern được cung cấp)
            if pattern and not re.search(pattern, full):
                continue

            tif_files.append(full)
    return tif_files

# --- Khối chạy chính để đếm file 2020 ---

root_dir = 'DATA_SV/ERA5' # Thư mục gốc chứa B04B, B05B, v.v.

# Pattern Regex để tìm chuỗi '2020' trong đường dẫn file
# Sử dụng re.escape để đảm bảo đường dẫn được xử lý đúng (nếu cần)
year_pattern = r'[\\/](2020)[\\/]'

# Gọi hàm để lấy danh sách file
files_2020 = list_all_files(root_dir, pattern=year_pattern)

# Kết quả
so_luong_file = len(files_2020)

print(f"Thư mục gốc đang tìm kiếm: {root_dir}")
print(f"Pattern lọc năm 2020: {year_pattern}")
print(f"\n✅ Số file .tif tìm thấy trong năm 2020 là: **{so_luong_file}**")

# Nếu bạn muốn xem một vài file đầu tiên:
# print("\nMột số file tìm thấy:")
# for f in files_2020[:5]:
#     print(f)

Thư mục gốc đang tìm kiếm: DATA_SV/ERA5
Pattern lọc năm 2020: [\\/](2020)[\\/]

✅ Số file .tif tìm thấy trong năm 2020 là: **29280**


In [2]:
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import glob
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import mapping

print(os.getcwd())
print(os.path.exists("VNM_ADM2.shp"))
print(os.path.exists("../VNM_ADM2.shp"))


def Endl():
    print('-----------------------------\n')


D:\UET_VNU\college years\3nd year college\semester 1\Artificial Intelligence\BTL_AI\rainfall_estimates
False
False


# **1. Lấy dữ liệu ranh giới các tỉnh ở Việt Nam**

In [4]:
# --- Thư mục chứa dữ liệu GeoTIFF và output CSV ---
root = "csv_data/Hima"
out_csv = 'csv_data/Hima/HIMA_hatinh.csv'

# --- Đường dẫn shapefile ranh giới ---
shp_path = "gadm41_VNM_shp"
vnm_gdf = gpd.read_file(shp_path)  # đọc shapefile cả Việt Nam

# THÊM DÒNG NÀY ĐỂ KIỂM TRA TÊN CỘT THUỘC TÍNH
print("Các cột có sẵn trong Shapefile:")
print(vnm_gdf.columns.tolist())
print(vnm_gdf.shape)
print(vnm_gdf.head())

Các cột có sẵn trong Shapefile:
['GID_1', 'GID_0', 'COUNTRY', 'NAME_1', 'VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1', 'ISO_1', 'geometry']
(63, 12)
     GID_1 GID_0  COUNTRY             NAME_1          VARNAME_1 NL_NAME_1  \
0  VNM.1_1   VNM  Vietnam           An Giang           An Giang        NA   
1  VNM.7_1   VNM  Vietnam  Bà Rịa - Vũng Tàu  Ba Ria - Vung Tau        NA   
2  VNM.3_1   VNM  Vietnam          Bắc Giang          Bac Giang        NA   
3  VNM.4_1   VNM  Vietnam            Bắc Kạn            Bac Kan        NA   
4  VNM.2_1   VNM  Vietnam           Bạc Liêu           Bac Lieu        NA   

  TYPE_1 ENGTYPE_1 CC_1 HASC_1  ISO_1  \
0   Tỉnh  Province   NA  VN.AG  VN-44   
1   Tỉnh  Province   NA  VN.BV     NA   
2   Tỉnh  Province   NA  VN.BG     NA   
3   Tỉnh  Province   NA  VN.BK     NA   
4   Tỉnh  Province   NA  VN.BL     NA   

                                            geometry  
0  POLYGON ((105.54862 10.42948, 105.54948 10.425...  
1  MULTIPOL

## **1.1. Lấy ranh giới tỉnh Hà Tĩnh**

In [5]:
# Lọc ra GeoDataFrame chỉ chứa tỉnh Hà Tĩnh
Ha_Tinh_gdf = vnm_gdf[vnm_gdf['VARNAME_1'] == 'Ha Tinh']
print(Ha_Tinh_gdf)


       GID_1 GID_0  COUNTRY   NAME_1 VARNAME_1 NL_NAME_1 TYPE_1 ENGTYPE_1  \
24  VNM.29_1   VNM  Vietnam  Hà Tĩnh   Ha Tinh        NA   Tỉnh  Province   

   CC_1 HASC_1 ISO_1                                           geometry  
24   NA  VN.HT    NA  MULTIPOLYGON (((106.3103 17.96227, 106.3057 17...  


In [9]:
# Lấy ra đối tượng hình học (geometry) ranh giới tỉnh Hà Tĩnh (chỉ 1 hàng)
ht_union = Ha_Tinh_gdf.geometry.iloc[0]
print(f"Ranh giới tỉnh Hà Tĩnh đã được lọc và sẵn sàng (Geometry Type: {ht_union.geom_type})")
Endl()
ht_crs = vnm_gdf.crs.to_string()  #lấy CRS
print(ht_crs)
Endl()
print(ht_union)

Ranh giới tỉnh Hà Tĩnh đã được lọc và sẵn sàng (Geometry Type: MultiPolygon)
-----------------------------

EPSG:4326
-----------------------------

MULTIPOLYGON (((106.31030273500005 17.962272644000052, 106.30570220900006 17.962369919000025, 106.30481720000012 17.959787368000093, 106.30188751300005 17.956024169000045, 106.2978439330002 17.953983308000034, 106.29627227700018 17.945611954000185, 106.29345703000001 17.943550110000103, 106.28919982800005 17.943014145000063, 106.28736877500012 17.939054489, 106.28687286400003 17.935531616000105, 106.28528595 17.936054231000128, 106.28479766900006 17.93403434800007, 106.28656768700012 17.92803383000006, 106.2911682130001 17.921209335000185, 106.29113006600005 17.920347215000106, 106.28673553500016 17.915552140000102, 106.28598785500009 17.91260910000011, 106.2836990350001 17.910394668000038, 106.27698516800012 17.91822052000009, 106.27497863700012 17.91961670000012, 106.27248382500011 17.91989898600002, 106.26990509100006 17.919000626000127

In [19]:
def extract_HaTinh_pixels_to_csv(path, output_csv, write_header):
    import rasterio
    import geopandas as gpd
    import numpy as np
    import pandas as pd
    from rasterio.features import geometry_mask
    from shapely.geometry import mapping

    with rasterio.open(path) as src:
        data = src.read(1).astype(float)
        nodata = src.nodata
        transform = src.transform
        src_crs = src.crs.to_string() if src.crs else None

    print(f"\nĐang xử lý file: {path}")
    print(f"Shape dữ liệu: {data.shape}, nodata: {nodata}, CRS: {src_crs}")

    if src_crs and src_crs != ht_crs:
        geom = gpd.GeoSeries([ht_union], crs=ht_crs).to_crs(src_crs).iloc[0]
        print(f"Đã reproject geometry Hà Tĩnh sang CRS raster: {src_crs}")
    else:
        geom = ht_union
        print("Geometry Hà Tĩnh CRS trùng CRS raster")

    rows_all, cols_all = np.meshgrid(np.arange(data.shape[0]), np.arange(data.shape[1]), indexing='ij')
    rows_flat = rows_all.ravel()
    cols_flat = cols_all.ravel()

    mask_flat = geometry_mask([mapping(geom)], invert=True, out_shape=data.shape, transform=transform).ravel()
    valid_pixels = mask_flat & (data.ravel() != nodata)

    print(f"Tổng số pixel trong raster: {data.size}")
    print(f"Số pixel thuộc Hà Tĩnh: {mask_flat.sum()}")
    print(f"Số pixel hợp lệ (khác nodata & trong Hà Tĩnh): {valid_pixels.sum()}")

    rows = rows_flat[valid_pixels]
    cols = cols_flat[valid_pixels]
    vals = data.ravel()[valid_pixels]
    lons, lats = rasterio.transform.xy(transform, rows, cols, offset='center')

    ts = pd.to_datetime(os.path.basename(path).split('_')[-1].split('.')[0], format='%Y%m%d%H%M%S')
    rel_path = os.path.relpath(path, root)
    var = rel_path.split(os.sep)[0]

    df = pd.DataFrame({
        'variable': var,
        'timestamp': ts,
        'row': rows,
        'col': cols,
        'lon': lons,
        'lat': lats,
        'value': vals
    })

    print(f"Số pixel sẽ ghi ra CSV: {len(df)}")
    print(df.head(5))

    df.to_csv(output_csv, mode='w' if write_header else 'a', index=False, header=write_header)
    return len(df)


In [20]:
def list_all_files(root):
    """
    Duyệt toàn bộ thư mục root và mọi thư mục con,
    trả về danh sách đường dẫn đầy đủ của tất cả file .tif.
    """
    tif_files = []

    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if f.lower().endswith(".tif"):
                full_path = os.path.join(dirpath, f)
                tif_files.append(full_path)

    return tif_files

In [21]:
from concurrent.futures import ThreadPoolExecutor

if __name__ == '__main__':
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)

    # Lấy danh sách toàn bộ file .tif trong thư mục gốc
    files = list_all_files(root)

    # Nếu CSV tồn tại, xóa để ghi mới
    if os.path.exists(out_csv):
        os.remove(out_csv)

    write_header = True  # chỉ ghi header ở file đầu tiên
    total = 0  # tổng số pixel hợp lệ đã ghi

    # Dùng đa luồng để tăng tốc
    with ThreadPoolExecutor(max_workers=4) as exe:
        futures = []

        # Gửi từng file vào pool
        for p in files:
            futures.append(
                exe.submit(extract_HaTinh_pixels_to_csv, p, out_csv, write_header)
            )

            # chỉ cho phép file đầu tiên được ghi header
            if write_header:
                write_header = False

        # Nhận kết quả dần khi các luồng hoàn thành
        for i, fut in enumerate(futures, 1):
            cnt = fut.result()
            total += cnt
            print(f"Processed {i}/{len(files)}: +{cnt:,} rows (total {total:,})")

    print("Hoàn thành flatten dữ liệu Hà Tĩnh!")


Hoàn thành flatten dữ liệu Hà Tĩnh!


In [15]:
print(os.getcwd())  # in ra thư mục hiện hành


D:\UET_VNU\college years\3nd year college\semester 1\Artificial Intelligence\BTL_AI\rainfall_estimates
